In [1]:
import sympy as sp
import numpy as np

In [2]:
z = sp.IndexedBase('z') # perturbation + correction

o = sp.symbols('o', cls=sp.Idx) # origin (source) of the signal/aberation
n = sp.symbols('n', cls=sp.Idx) # channel number

theta = sp.symbols(r'\theta', real=True) # angle of the source

In [3]:
def set_noise(M, phase_rms=0, amplitude_rms=0):
    for i in ['i', 'n', 's']:
        if i == 'i':
            rng = range(1, 5)
        elif i == 'n':
            rng = range(0, 4)
        elif i == 's':
            rng = range(0, 7)

        phase_perturbations = np.random.normal(0, phase_rms, len(rng)+1)
        amplitude_perturbations = np.random.normal(1, amplitude_rms, len(rng)+1)
        amplitude_perturbations /= np.mean(amplitude_perturbations)
        for j in rng:
            if i == 'i':
                M = M.subs(z[i,j], amplitude_perturbations[j] * sp.exp(sp.I * phase_perturbations[j]))
            else:
                M = M.subs(z[i,j], 1)
    return M

<div align=center>

---

# Elements definition

</div>

4 input beams

In [4]:
x = sp.IndexedBase('x') # input signal
X = sp.Matrix([
    [x[1]],
    [x[2]],
    [x[3]],
    [x[4]]
])
X

Matrix([
[x[1]],
[x[2]],
[x[3]],
[x[4]]])

Atmospherical perturbation

In [5]:
Zi = sp.Matrix([
    [z['i',1], 0, 0, 0],
    [0, z['i',2], 0, 0],
    [0, 0, z['i',3], 0],
    [0, 0, 0, z['i',4]]
])
Zi

Matrix([
[z[i, 1],       0,       0,       0],
[      0, z[i, 2],       0,       0],
[      0,       0, z[i, 3],       0],
[      0,       0,       0, z[i, 4]]])

Nuller

In [6]:
N = 1/sp.sqrt(4) * sp.Matrix([
    [1, 1, 1, 1],
    [1, 1,-1,-1],
    [1,-1, 1,-1],
    [1,-1,-1, 1],
])
N

Matrix([
[1/2,  1/2,  1/2,  1/2],
[1/2,  1/2, -1/2, -1/2],
[1/2, -1/2,  1/2, -1/2],
[1/2, -1/2, -1/2,  1/2]])

Check if the nuller is physical

In [7]:
sp.conjugate(N).T * N

Matrix([
[1, 0, 0, 0],
[0, 1, 0, 0],
[0, 0, 1, 0],
[0, 0, 0, 1]])

Nuller perturbations

In [8]:
Zn = sp.Matrix([
    [z['n',0], 0, 0, 0],
    [0, z['n',1], 0, 0],
    [0, 0, z['n',2], 0],
    [0, 0, 0, z['n',3]]
])
Zn

Matrix([
[z[n, 0],       0,       0,       0],
[      0, z[n, 1],       0,       0],
[      0,       0, z[n, 2],       0],
[      0,       0,       0, z[n, 3]]])

Nuller operations

In [9]:
nuller_output = Zn * N * X
sp.Matrix([y.factor() for y in nuller_output])

Matrix([
[(x[1] + x[2] + x[3] + x[4])*z[n, 0]/2],
[(x[1] + x[2] - x[3] - x[4])*z[n, 1]/2],
[(x[1] - x[2] + x[3] - x[4])*z[n, 2]/2],
[(x[1] - x[2] - x[3] + x[4])*z[n, 3]/2]])

In [10]:
S = 1/sp.sqrt(4) * sp.Matrix([
    [sp.sqrt(4), 0, 0, 0],
    [0, 1, sp.exp(sp.I * theta), 0],
    [0, -sp.exp(-sp.I * theta), 1, 0],
    [0, 1, 0, sp.exp(sp.I * theta)],
    [0, -sp.exp(-sp.I * theta), 0, 1],
    [0, 0, 1, sp.exp(sp.I * theta)],
    [0, 0, -sp.exp(-sp.I * theta), 1],
])
S

Matrix([
[1,                 0,                 0,               0],
[0,               1/2,   exp(I*\theta)/2,               0],
[0, -exp(-I*\theta)/2,               1/2,               0],
[0,               1/2,                 0, exp(I*\theta)/2],
[0, -exp(-I*\theta)/2,                 0,             1/2],
[0,                 0,               1/2, exp(I*\theta)/2],
[0,                 0, -exp(-I*\theta)/2,             1/2]])

In [11]:
sp.conjugate(S).T * S

Matrix([
[1, 0, 0, 0],
[0, 1, 0, 0],
[0, 0, 1, 0],
[0, 0, 0, 1]])

In [12]:
Zs = sp.Matrix([
    [1, 0, 0, 0, 0, 0, 0],
    [0, z['s',1], 0, 0, 0, 0, 0],
    [0, 0, z['s',2], 0, 0, 0, 0],
    [0, 0, 0, z['s',3], 0, 0, 0],
    [0, 0, 0, 0, z['s',4], 0, 0],
    [0, 0, 0, 0, 0, z['s',5], 0],
    [0, 0, 0, 0, 0, 0, z['s',6]],
])
Zs

Matrix([
[1,       0,       0,       0,       0,       0,       0],
[0, z[s, 1],       0,       0,       0,       0,       0],
[0,       0, z[s, 2],       0,       0,       0,       0],
[0,       0,       0, z[s, 3],       0,       0,       0],
[0,       0,       0,       0, z[s, 4],       0,       0],
[0,       0,       0,       0,       0, z[s, 5],       0],
[0,       0,       0,       0,       0,       0, z[s, 6]]])

In [13]:
K = sp.Matrix([
    [1, 0, 0, 0, 0, 0, 0],
    [0, 1,-1, 0, 0, 0, 0],
    [0, 0, 0, 1,-1, 0, 0],
    [0, 0, 0, 0, 0, 1,-1],
])
K

Matrix([
[1, 0,  0, 0,  0, 0,  0],
[0, 1, -1, 0,  0, 0,  0],
[0, 0,  0, 1, -1, 0,  0],
[0, 0,  0, 0,  0, 1, -1]])

In [14]:
M = K * Zs * S * Zn * N * Zi
M

Matrix([
[                                                                                           z[i, 1]*z[n, 0]/2,                                                                                             z[i, 2]*z[n, 0]/2,                                                                                             z[i, 3]*z[n, 0]/2,                                                                                             z[i, 4]*z[n, 0]/2],
[((exp(I*\theta)*z[s, 1]/2 - z[s, 2]/2)*z[n, 2]/2 + (z[s, 1]/2 + exp(-I*\theta)*z[s, 2]/2)*z[n, 1]/2)*z[i, 1], (-(exp(I*\theta)*z[s, 1]/2 - z[s, 2]/2)*z[n, 2]/2 + (z[s, 1]/2 + exp(-I*\theta)*z[s, 2]/2)*z[n, 1]/2)*z[i, 2],  ((exp(I*\theta)*z[s, 1]/2 - z[s, 2]/2)*z[n, 2]/2 - (z[s, 1]/2 + exp(-I*\theta)*z[s, 2]/2)*z[n, 1]/2)*z[i, 3], (-(exp(I*\theta)*z[s, 1]/2 - z[s, 2]/2)*z[n, 2]/2 - (z[s, 1]/2 + exp(-I*\theta)*z[s, 2]/2)*z[n, 1]/2)*z[i, 4]],
[((exp(I*\theta)*z[s, 3]/2 - z[s, 4]/2)*z[n, 3]/2 + (z[s, 3]/2 + exp(-I*\theta)*z[s, 4]/2)*z[n, 1]/2)

In [15]:
Y = M * X
Y

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                              x[1]*z[i, 1]*z[n, 0]/2 + x[2]*z[i, 2]*z[n, 0]/2 + x[3]*z[i, 3]*z[n, 0]/2 + x[4]*z[i, 4]*z[n, 0]/2],
[(-(exp(I*\theta)*z[s, 1]/2 - z[s, 2]/2)*z[n, 2]/2 - (z[s, 1]/2 + exp(-I*\theta)*z[s, 2]/2)*z[n, 1]/2)*x[4]*z[i, 4] + (-(exp(I*\theta)*z[s, 1]/2 - z[s, 2]/2)*z[n, 2]/2 + (z[s, 1]/2 + exp(-I*\theta)*z[s, 2]/2)*z[n, 1]/2)*x[2]*z[i, 2] + ((exp(I*\theta)*z[s, 1]/2 - z[s, 2]/2)*z[n, 2]/2 - (z[s, 1]/2 + exp(-I*\theta)*z[s, 2]/2)*z[n, 1]/2)*x[3]*z[i, 3] + ((exp(I*\theta)*z[s, 1]/2 - z[s, 2]/2)*z[n, 2]/2 + (z[s, 1]/2 + exp(-I*\theta)*z[s, 2]/2)*z[n, 1]/2)*x[1]*z[i, 1]],
[(-(exp(I*\theta)*z[s, 3]/2 - z[s, 4]/2)*z[n, 3]/2 - (z[s

In [16]:
sp.Matrix([sp.simplify(y * sp.conjugate(y)) for y in Y])

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

<div align=center>

---

# Application

</div>

In [17]:
output = sp.simplify(Y.subs(theta, sp.pi/2))

In [18]:
for i in X:
    output = output.subs(i, 1)
output

Matrix([
[                                                                                                                                                                                                                                                             (z[i, 1] + z[i, 2] + z[i, 3] + z[i, 4])*z[n, 0]/2],
[-((I*z[s, 1] - z[s, 2])*z[n, 2] - (z[s, 1] - I*z[s, 2])*z[n, 1])*z[i, 2]/4 + ((I*z[s, 1] - z[s, 2])*z[n, 2] - (z[s, 1] - I*z[s, 2])*z[n, 1])*z[i, 3]/4 + ((I*z[s, 1] - z[s, 2])*z[n, 2] + (z[s, 1] - I*z[s, 2])*z[n, 1])*z[i, 1]/4 - ((I*z[s, 1] - z[s, 2])*z[n, 2] + (z[s, 1] - I*z[s, 2])*z[n, 1])*z[i, 4]/4],
[-((I*z[s, 3] - z[s, 4])*z[n, 3] - (z[s, 3] - I*z[s, 4])*z[n, 1])*z[i, 2]/4 + ((I*z[s, 3] - z[s, 4])*z[n, 3] - (z[s, 3] - I*z[s, 4])*z[n, 1])*z[i, 4]/4 + ((I*z[s, 3] - z[s, 4])*z[n, 3] + (z[s, 3] - I*z[s, 4])*z[n, 1])*z[i, 1]/4 - ((I*z[s, 3] - z[s, 4])*z[n, 3] + (z[s, 3] - I*z[s, 4])*z[n, 1])*z[i, 3]/4],
[-((I*z[s, 5] - z[s, 6])*z[n, 3] - (z[s, 5] - I*z[s, 6])*z[n, 2])*z[i, 3]

In [19]:
output = sp.simplify(set_noise(output, phase_rms=0.001, amplitude_rms=0.001))
output

Matrix([
[                                                                                                                  2.00013809542317 - 0.000157797304853144*I],
[               (0.99999904341041 - 0.00138317687428144*I)*(0.500471234993092 - 0.500471234993092*I + 0.500302235107101*(-1 + I)*exp(0.00204172135082958*I))],
[(0.999998289451195 - 0.00184962014593246*I)*(0.500471234993092*(1 - I)*exp(0.000466443885229215*I) + 0.500088324639149*(-1 + I)*exp(0.00138317731532581*I))],
[             (0.999999891215053 - 0.000466443868315191*I)*(-0.500088324639149 + 0.500302235107101*(1 - I)*exp(0.00112498792073298*I) + 0.500088324639149*I)]])

In [20]:
intensities = sp.Matrix([ sp.Abs(i)**2 for i in output ])
intensities

Matrix([
[                                                                                                                         4.00055242566303],
[                            1.00154756701734 - 0.50077375494771*exp(0.00204172135082958*I) - 0.50077375494771*exp(-0.00204172135082958*I)],
[1.0011195789918 - 0.500559642875562*exp(0.000916733430096597*I) - 5.55111512312578e-17*I - 0.500559642875562*exp(-0.000916733430096597*I)],
[                           1.0007813177871 - 0.500390613135863*exp(0.00112498792073298*I) - 0.500390613135863*exp(-0.00112498792073298*I)]])

In [21]:
output_energy = np.sum(intensities)
output_energy

7.00400088945928 - 0.50077375494771*exp(0.00204172135082958*I) - 0.500390613135863*exp(0.00112498792073298*I) - 0.500559642875562*exp(0.000916733430096597*I) - 5.55111512312578e-17*I - 0.500559642875562*exp(-0.000916733430096597*I) - 0.500390613135863*exp(-0.00112498792073298*I) - 0.50077375494771*exp(-0.00204172135082958*I)